# Preconditioned Conjugate Gradient Algorithm on Toeplitz Matrix Computation

# Aurthor: Xiangyu Wang

# Instructor: Dr. Haomin Zhou

## Project Inroduction

In computational mathematics, people define iterative method as a mathematical procedure that uses an initial value to generate a sequence of improving approximate solutions for a class of problems, in which the n-th approximation is derived from the previous ones. Iterative method also contains terminating criteria, which is designed by different purposes. $10^{-6}$ is usually a reasonal terminating critrion. Iterative method used for getting convergency in order to abate the direct method's computational complexity. In terms of applying iterative method into numerical linear algebra, we can implement matrix vector multiplications way faster than directly solve them. $CG$ algorithm a one of classic iteration methods. The main idea of $CG$ is to reduce the residual by iterations and get a convergence. 

Preconditioners are used to decrease the condition number in order to make the iteration number independant of matrix size. There are different ways to deisng preconditioners. In this project, we area dealing with a special case. We would have toeplitz matrix as the multiplier, a toeplitz matrix contains constant on the diagonal. In addition, we will design circulant matrix as our preconditioner. In the end, according to the property of circulant matrix, it is able to design a fast fourier transform of the first column to improve our $CG$ algorithm in another level.

The details will be expressed in the following section. 

## Python jupyter notebook introduciton

To test the code, we need to have jupyter notebook and Python 3.0 or newer version installed in our PC.

There are few libraries used in this project. We have to make sure they are installed in PC before we run the following cell. We can do this in Terminal.

To run the code, we just select the code cell and hit run on the tool bar. Then we can run the code. Make sure we run it in correct order so it could store correct data.

Here are the libraries involved in this project.


In [368]:
import numpy as np
import math
from numpy.linalg import inv
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sympy import *
from scipy.integrate import quad
import scipy.fft
import time

## CG & PCG algorithm

According to the lecture, the algorithm of $CG$ algorithm is coded up as following:

Remark: the termination criteria is $\frac{\|r_k\|}{\|r_0\|} \leq 10^{-6}$ as stated in the project statement.

In [154]:
def conjugate_grad(A, b, it = 10000):
    n = A.shape[0]
    x = np.zeros(n)
    r = b - np.dot(A, x)
    r0 = r.copy()
    p = r.copy()
    r_old = np.inner(r, r)
    rk_norm = np.linalg.norm(r)
    for it in range(it):
        q = np.dot(A,p)
        alpha = r_old / np.inner(p, q) 
        x += alpha * p 
        r -= alpha * q 
        r_new = np.inner(r, r) 
        rk_norm = np.linalg.norm(r)
        r0_norm = np.linalg.norm(r0)
        if rk_norm / r0_norm <= 1e-6:
            break
        beta = r_new / r_old 
        p = r + beta * p 
        r_old = r_new.copy() # Update the inner_product of the r inner product
    return x, it+1

According to the lecture, the algorithm of $PCG$ algorithm is coded up as following:

Compared to $CG$, $PCG$ has conditioner within and there are more matrix vector multiplication needed to be dealt with later on.

Remark: the termination criteria is $\frac{\|r_k\|}{\|r_0\|} \leq 10^{-6}$ as stated in the project statement.

In [155]:
# 2 fft and 2 inverse fft
def pcg(A, b, M, it = 10000):
    x = np.zeros(n)
    r = b - A @ x#
    r0 = r.copy()
    z = np.zeros(n)
    z_bef = inv(M) @ r
    p = np.squeeze(z_bef.copy())
    for i in range(n):
        r_bef = r
        q = A @ p
        alpha = (r_bef @ z_bef)/(p @ q)
        x = x + alpha*p
        r = r_bef - alpha*q
        z = inv(M) @ r
        if (np.linalg.norm(r)/np.linalg.norm(r0)) < 1e-6:
            break
        else:
            beta = np.dot(r,z)/np.dot(r_bef,z_bef)
            p = np.squeeze(z + beta*p)
            z_bef = np.squeeze(z)

    return x, i+1

If you read till here, I suggest read the $PCG$ $FFT$ $algorithm$ part at the end because to accomplish that we need to introduce toeplitz matrix and circulant matrix before. I will clearly express the way I design these $2$ matrixes in the following portions.

## PCG_FFT  algorithm

Welcome back! Now we are clear about the formation of toeplitz and circulant matrices. As we stated in the first portion, in this portion we will use the property of circulant matrices and modify our previous PCG method.

Matrix vector multiplication will be extremely expensive when we have large dataset. In the PCG method, we have 4 matrix vector multiplication, two of them containing a inverse multiplication. 

If a matrix is circulant, then we have $C_n$ = $F_n^*$$diag(\sqrt n F_n)F_n$, here, $F_n$ is unitary DFT matrix.

$[F]_{j,k}$ = $\frac{1}{\sqrt n}e^{\frac{2\pi ijk}{n}}$

And the diagonal matrix with main diagonal elements. We can get to $\frac{1}{\sqrt N}F_n^*x$ can be calcuted by scipy.fft.ifft(x). Further, if we decomposite $C(x)y$, this is equivalent to ifft(fft(x).*fft(y)). According to this, we can design our FFT for the matrix vector multiplication.

Now, if it is inverse matrix vector multiplication, we have 

$C^{-1}(x)$ = $(F_n^*$$diag(\sqrt n F_n x)F_n)^{-1}$

= $F_n^{-1}D^{-1}(\sqrt n F_nx)F_n$

We can use ifft(1/fft(x)) to express this.

Therefore we get, $C^{-1}(x)y$ = ifft(fft(y)/fft(x))

In our case, we would only use the first column of the circulant matrix.

So we can design the FFT for the preconditioner mulitiplication becuase they are circulant. And we also need to transform toeplitz matrices to circulant matrixes. As stated in the project description, we will increase the A size as $2xn$ and embed a matrix into it. According to Dr. Paulo Jorge's paper, the embedding is designed as 

An arbitrary Toeplitz matrix matrix $T \in M_n$ with elements

$T_{ik}$ = $t_{i-k}$ $( 0\leq i$, $k)$ can be embedded in a circulant C $\in M_m$ if $m\geq 2n-1$. In general, this leads to a matrix $C$ which will contain $l$ = $m - 2n +1$ arbitrary parameters $\alpha_i$. THe embedding is defined by the elements $c_i$ of the first row of $C$,

$C_i = t_i$  when $0\leq i\leq n$

$C_i = \alpha_{i-n}$  when $n\leq i\leq n+l$

$C_i = t_{i-m}$  when $n+l\leq i\leq m$

Here is my implenmetation:


In [156]:
def A_2n(A):
    n = A.shape[0]
    first_column = A[:,0]
    A_new_1 = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            for k in range(1,n):
                if j == i+k:
                    A_new_1[i][j] = first_column[n-k]
                if j == i-k:
                    A_new_1[i][j] = first_column[n-k]
    A_new = np.zeros((2*n,2*n))
    A_new[0:n,0:n] = A
    A_new[n:2*n,n:2*n] = A
    A_new[n:2*n,0:n] = A_new_1
    A_new[0:n,n:2*n] = A_new_1
    
    return A_new

When I try this matrix, it is very slow because it requires $O(n^3)$, becuase we would only need the first column,

I creat a $O(n)$ function just return the first column as following, this increases my speed a lot.

In [216]:
def A_2n_fast(A):
    n = A.shape[0]
    A_new = np.zeros(2*n)
    first_col = A[:,0]
    A_new[0:n] = first_col
    first_col = first_col[1:n] 
    A_new[n+1:2*n] = first_col[::-1]
    return A_new

In [177]:
def b_2n(b):
    b_new = np.zeros(2*n)
    b_new[0:n] = b
    return b_new

According to the method I described above, I replaced four matrix vector multiplication as following:

To match increased $A$, I put zeros into $p$ vector to match the dimension. In this case we will have a $2n$ vector as our result and we just take the first $n$ and take the real value.

In [399]:
def pcg_fft(A, n, A_increase, b, M, it = 10000):
    x = np.zeros(n)
#     r = b - A @ x#
    r = b - np.real(scipy.fft.ifft(scipy.fft.fft(A[:,0]) *  scipy.fft.fft(x)))
    r0 = r.copy()
    z = np.zeros(n)
    # z_bef = inv(M) @ r #(inverse)
    z_bef = np.real(scipy.fft.ifft(scipy.fft.fft(r) / scipy.fft.fft(M)))
    p = z_bef.copy()
    for i in range(it):
        r_bef = r
        p_temp = np.r_[p,np.zeros(n)]
        q = np.real(scipy.fft.ifft(scipy.fft.fft(A_increase) * scipy.fft.fft(p_temp))[0:n])
#         q = A @ p
        alpha = (r_bef @ z_bef)/(p @ q)
        x = x + alpha*p
        r = r_bef - alpha*q
#         z = inv(M) @ r#inverse
        z = np.real(scipy.fft.ifft(scipy.fft.fft(r) / scipy.fft.fft(M)))
        if (np.linalg.norm(r)/np.linalg.norm(r0)) < 1e-6:
            break
        else:
            beta = np.dot(r,z)/np.dot(r_bef,z_bef)
            p = z + beta*p
#             p_temp[0:n] = p
            z_bef = z
    return x, i+1

## Form Toeplitz matrix

In this portion, I will describe the way I design toeplitz matrix. There are $2$ different toeplitz matrices.

### First toeplitz system
First toeplitz matrix is given by function $a_{k} = \|k+1\|^{-p}$ for the lower triangular part of $A_{n}.$

According to the index of $A_n$, if $a_n$ is on lower triangular places, the row index of $a_n$ = span{i-1,i-2,...i-(n-1)}, where $i$ is the row index and the minus term will be the column index. So we can representate the index of $a$ as $i$ - $j$. In addition, we can express lower triangular places as $j$ < $i$. After this we just need to make A symmetric. The way I did was $A$ = $A$ + $A^T$ - $diag(A)$

So the first toeplitz system formation will be as following:

Here $n$ signifies the size of matrix and $p$ is the expotential order.


In [329]:
def toeplitz_1(n, p):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if j < i:
                A[i, j] = abs(i - j + 1)**(-p)
            elif i == j:
                A[i, j] = 1
    # Make A as symmetric
    A += A.T - np.diag(A.diagonal())
    return A
    

In [350]:
print('Simple example of first toeplitz system when n=3, p=0.1:\n',toeplitz_1(3, 1/10))

Simple example of first toeplitz system when n=3, p=1:
 [[1.         0.93303299 0.89595846]
 [0.93303299 1.         0.93303299]
 [0.89595846 0.93303299 1.        ]]


In [354]:
print('Eigenvalue of this matrix = \n',np.linalg.eig(toeplitz_1(3, 1/10))[0])

Eigenvalue of this matrix = 
 [2.84145923 0.10404154 0.05449923]


We can see that the eigenvalues when $p$<=$1$ are still positive, and $A$ is symmetric as designed. We can conclude that $A$ is still SPD if $p$<=$1$

### Second toeplitz system

This method is based on Fast Fourier Transform$(FFT)$. As stated, $A_k = \frac{1}{2\pi}\int^\pi_{-\pi} f(\theta)e^{-ik\theta}d\theta$, $k$ = $0$, $-1$, $1$,... this is just as $A$ matrix index $i$-$j$.

$f(\theta)$ = $\theta^4$ + $1$ for $\theta \in(-\pi,\pi)$. 

My first attempt was to integrate this function directly. I tried this method at the beginning and if the size of matrix is large, it will take forever to run becuae it has $O(n^3)$ as computational complexity.

Here would be my first attempt


In [332]:
def integrand(theta):  # a_0
    return theta**4 + 1
def integrand_2(theta,k):  # a_n
    return (theta**4 + 1) * np.cos(k*theta)
def integrand_3(theta,k):  # b_n, but this always equal to zero, not gonna use
    return (theta**4 + 1) * np.sin(k*theta)

def toeplitz_2_2(n):
    A = np.zeros(2*n-1)
    for i in range(n):
        for j in range(n):
            if i == j:
                temp = quad(integrand, -math.pi, math.pi)
                A[0] = (temp[0]/2/math.pi)
            else:
                k = i-j
                temp = quad(integrand_2, -math.pi, math.pi, args=(k))
                if i>j:
                    A[k] = (temp[0]/2/math.pi)
                elif i<j:
                    A[n-k-1] = (temp[0]/2/math.pi)
    Matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                Matrix[i][j] = A[0]
            elif i>j:
                Matrix[i][j] = A[i-j]
            elif i<j:
                Matrix[i][j] = A[n-i+j-1]
    return Matrix

Then according to the project statement, I figured out the fast way to get this toeplitz matrix.
First we will make $f(\theta)$ as a vector for $\theta \in (-\pi,\pi)$. Here I picked 10000 discretized point to make it more accurate.

Then we need to take fourier transform of this vector and just take the real part, and divided by the lenth of the vector. Here is $10000$. Till now we obtained a $10000$x$1$ vector.

After this we can get our toeplitz column by taking the first $n$ values of this vector. Further, we make the odd index negtive as it is before. Finally we toeplitz this vector, we will get our toeplitz matirx. 

This one has $O(nlogn)$ complexity, which is very fast forming a large matrix.

My implentament and simple example will be shown as following:

In [348]:
def toeplitz_b(n):
    theta = np.linspace(-np.pi,np.pi,10000)
    f = (theta**4 +1)
    col = np.real(np.fft.fft(f))/len(f)
    ak = col[0:n]
    ak[1:][::2] = -ak[1:][::2]
    return scipy.linalg.toeplitz(ak)

In [349]:
print('Simple example for sencond toeplitz matirx when n=3:\n',toeplitz_b(3))

Simple example for sencond toeplitz matirx when n=3:
 [[ 20.48961223 -15.48460911   8.37295078]
 [-15.48460911  20.48961223 -15.48460911]
 [  8.37295078 -15.48460911  20.48961223]]


## Preconditioner building

In this portion, I will detailed express my implentation forming G_strang and T_chan preconditioner.

### G_Strang's circulant preconditioner

$c_j = a_j$ if $0\leq j\leq [n/2]$

$c_j = a_{j-n}$ if $[n/2] \leq j$ < $n$

$c_j = a_{n+j}$ if $0$ < $-j$ < $n$

$[x]$ signifies the closest integer(smaller) than $x$.

To implement this, I created a function that could transform $k$ index into location for $A$ matrix. This problem is basically a index-location transformation. My method has $O(n^2)$ as I create a loop within loop.

Here is how I implemment this:

In [123]:
def G_strang(n, A):
    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if j < i:
                k = -n + i - j
            else:
                k = i - j 
            if 0 <= k <= math.floor(n/2):
                kx, ky = A_matrix_index(k, n)
                C[i, j] = A[kx, ky]
            elif  math.floor(n/2) < k < n:
                k = k - n
                kx, ky = A_matrix_index(k, n)
                C[i, j] = A[kx, ky]
            elif 0 < -k < n:
                k = n + k
                if math.floor(n/2) < k < n:
                    k = k - n
                kx, ky = A_matrix_index(k, n)
                C[i, j] = A[kx, ky]            
    return C

In [355]:
print('Simple example of G_Strang preconditioner:\n',G_strang(3,toeplitz_1(3,2)))

Simple example of G_Strang preconditioner:
 [[1.   0.25 0.25]
 [0.25 1.   0.25]
 [0.25 0.25 1.  ]]


This G_strang_fast function is used to return the first column instead of the whole matrix, as we can see below, it is only $O(n)$ because I set j is always 0. In this case, if we need a big size matrix, this will be way faster because in the FFT, we are going to take only the first column, which I will state in the CG_FFT portion.

In [339]:
def G_strang_fast(n, A):
    C = np.zeros(n)
    for i in range(n):
        j = 0 # Difference: here, j is always 0.
        if j < i:
            k = -n + i - j
        else:
            k = i - j 
        if 0 <= k <= math.floor(n/2):
            kx, ky = A_matrix_index(k, n)
            C[i] = A[kx, ky]
        elif  math.floor(n/2) < k < n:
            k = k - n
            kx, ky = A_matrix_index(k, n)
            C[i] = A[kx, ky]
        elif 0 < -k < n:
            k = n + k
            if math.floor(n/2) < k < n:
                k = k - n
            kx, ky = A_matrix_index(k, n)
            C[i] = A[kx, ky]            
    return C

In [357]:
print('Same example:',G_strang_fast(3,toeplitz_1(3,2)))

Same example: [1.   0.25 0.25]


This is the index-location transformation for $A$ matrix as I stated before. Here, we need the index $k$ and the matrix size $n$ in order to return the location.

My function is given as following:

In [124]:
def A_matrix_index(k, n):
    if k <= 0:
    # If k is less than or equal to 0, we could only consider the first row of A matrix to get it's location.
        kx = 0
        ky = -k
    elif k > 0:
    # If k is greater than 0, we could only consider the first column of A matrix to get it's location.
        kx = k
        ky = 0
    # This is in T_chan's case if k - n = n or -n in this case we use k = 0
    if k == n or k == -n:
        kx = ky = 0
    return kx, ky

### T_Chan's circulant preconditioner

Similarly with G_Strang's preconditioner, we have T_chan's preconditioner:

$C_j$ = $\frac{(n-j)a_j+ja_{j-n}}{n}$, if $0 \leq j \leq n$

$C_j$ = $c_{n+j}$, if $0$ <  $-j$ < $n$

It has the same complexity with G_Strang's preconditioner, $O(n^2)$. So it has the same issue when we require a giant size matrix transformation.

In this case, we also need a fast T_chan matrix to get the first column and make the transformation faster.

In [125]:
def T_chan(n, A):
    C = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if j < i:
                k = -n + i - j
            else:
                k = i - j
            if 0 <= k < n:
                kx1, ky1 = A_matrix_index(k, n)
                kx2, ky2 = A_matrix_index(k-n, n)
                C[i, j] = ((n-k)*A[kx1, ky1] + k*A[kx2, ky2])/n
            else:
                k = k + n
                kx1, ky1 = A_matrix_index(k, n)
                kx2, ky2 = A_matrix_index(k-n, n)
                C[i, j] = ((n-k)*A[kx1, ky1] + k*A[kx2, ky2])/n
    
    return C

In [359]:
print('Simple example of T_chan with same condition as G_strang:\n',T_chan(3,toeplitz_1(3,2)))

Simple example of T_chan with same condition as G_strang:
 [[1.        0.2037037 0.2037037]
 [0.2037037 1.        0.2037037]
 [0.2037037 0.2037037 1.       ]]


This would be the T_chan_fast function returning the first column of the preconditioner matrix.

Similarly, it is only $O(n)$ because $j$ is set as $0$.

In [345]:
def T_chan_fast(n, A):
    C = np.zeros(n)
    for i in range(n):
        j = 0
        if j < i:
            k = -n + i - j
        else:
            k = i - j
        if 0 <= k < n:
            kx1, ky1 = A_matrix_index(k, n)
            kx2, ky2 = A_matrix_index(k-n, n)
            C[i] = ((n-k)*A[kx1, ky1] + k*A[kx2, ky2])/n
        else:
            k = k + n
            kx1, ky1 = A_matrix_index(k, n)
            kx2, ky2 = A_matrix_index(k-n, n)
            C[i] = ((n-k)*A[kx1, ky1] + k*A[kx2, ky2])/n
    
    return C

In [361]:
print('Simple example of T_chan with same condition as G_strang:\n',T_chan_fast(3,toeplitz_1(3,2)))

Simple example of T_chan with same condition as G_strang:
 [1.        0.2037037 0.2037037]


Now it is a good time to go back and read the CG_FFT algorithm

## Results and Comments

I picked $b$ matrix as $1$ at the first place and $0$s at other places. Now let's count the CPU time

### First topelize system

### CG method with toeplitz system A

In [405]:
# Regular CG method
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:
    
    b = np.zeros(n)
    b[0] = 1
    A1 = toeplitz_1(n,p[0])
    start = time.time()
    result,iteration = conjugate_grad(A1,b)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'CG time spend for n = {n}, first toeplitz matrix :', end - start,'s')
    else:
        print(f'CG time spend for n = {n}, first toeplitz matrix :', end - start,'s')

[ 1.07071523e+00 -2.51649958e-01 -4.80171812e-02 -2.19600584e-02
 -1.28180240e-02 -8.47300072e-03 -6.04629146e-03 -4.54474684e-03
 -3.54795233e-03 -2.85082386e-03 -2.34320723e-03 -1.96157661e-03
 -1.66713922e-03 -1.43505532e-03 -1.24878258e-03 -1.09694076e-03
 -9.71485876e-04 -8.66599886e-04 -7.77990075e-04 -7.02433030e-04
 -6.37469928e-04 -5.81198361e-04 -5.32127332e-04 -4.89074475e-04
 -4.51092086e-04 -4.17413082e-04 -3.87411023e-04 -3.60570137e-04
 -3.36462608e-04 -3.14731170e-04 -2.95075632e-04 -2.77242367e-04
 -2.61016037e-04 -2.46213042e-04 -2.32676314e-04 -2.20271176e-04
 -2.08882067e-04 -1.98410000e-04 -1.88770684e-04 -1.79893316e-04
 -1.71720139e-04 -1.64207105e-04 -1.57326322e-04 -1.51071895e-04
 -1.45473013e-04 -1.40624981e-04 -1.36773831e-04 -1.34597962e-04
 -1.36601121e-04 -1.69699096e-04] 12
CG time spend for n = 50, first toeplitz matrix : 0.0011150836944580078 s
CG time spend for n = 400, first toeplitz matrix : 0.0013210773468017578 s
CG time spend for n = 800, first t

Here I did not loop in different $p$ to better show the result, and I just show $n$ = $50$'s result to compare the result with $PCG$ to make sure if they are right. Large n I just gave time consuming. We can modify $p$ value to test the result but it will not impact the speed. The most time spend will be forming the $A$ mtraix and $b$ matrix. We can also see that the time consuming in $CG$ is becoming large if we hit a large number, and it will increase expotentially.

### CG method for toeplitz system B(FFT version)

In [406]:
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:

    b = np.zeros(n)
    b[0] = 1
    A1 = toeplitz_b(n)
    start = time.time()  
    result,iteration = conjugate_grad(A1,b)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'CG time spend for n = {n}, second toeplitz matrix :', end - start,'s')
    else:
        print(f'CG time spend for n = {n}, second toeplitz matrix :', end - start,'s')

[ 1.36698293e-01  1.51232063e-01  7.56490781e-02  2.22610645e-02
 -3.07095815e-03 -6.80922815e-03 -5.00473737e-03 -1.61258291e-03
 -3.69423174e-04  4.17313430e-04  1.54857867e-04  2.21577732e-04
 -4.70554347e-05  6.16414300e-05 -7.32630674e-05  4.49778501e-05
 -4.99364188e-05  4.20838659e-05 -3.73729721e-05  3.47865152e-05
 -3.11350124e-05  2.84732256e-05 -2.62451990e-05  2.40919132e-05
 -2.21628896e-05  2.06960467e-05 -1.91225784e-05  1.77760338e-05
 -1.67465852e-05  1.55549722e-05 -1.45847151e-05  1.38314420e-05
 -1.29238867e-05  1.21791013e-05 -1.16229435e-05  1.09342105e-05
 -1.03266793e-05  9.90684224e-06 -9.39903034e-06  8.89954529e-06
 -8.52494819e-06  8.22123230e-06 -7.82310480e-06  7.27152106e-06
 -7.70620349e-06  6.42037494e-06 -6.52256861e-06  9.33720083e-06
  1.25142483e-06  1.91989311e-05] 42
CG time spend for n = 50, second toeplitz matrix : 0.0044002532958984375 s
CG time spend for n = 400, second toeplitz matrix : 0.0035660266876220703 s
CG time spend for n = 800, secon

This is the FFT toeplitz matrix, it is very fast because we used a fast way to form the matrix. It could match the result we got from last version that most of the time spent is making the matrix. I tried my previous integral version, that one will take forever to run if we have large n because it is  $O(n^3)$ as stated before.

## PCG for toeplitz A in G_strang conditioner

In [408]:
# Regular CG method
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:
   
    b = np.zeros(n)
    b[0] = 1
    A = toeplitz_1(n,p[0])
    A_increase = A_2n_fast(A)
    C = G_strang_fast(n,A)
    start = time.time()
    result,iteration = pcg_fft(A,n,A_increase,b,C)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'G_strang PCG time spend for n = {n}, first toeplitz matrix :', end - start,'s')
    else:
        print(f'G_strang PCG time spend for n = {n}, first toeplitz matrix :', end - start,'s')
# result, iteration = pcg_fft(test_b,n,A_increase,b,C)
# # print(iteration)
# print(result)

[ 1.07071523e+00 -2.51649957e-01 -4.80171813e-02 -2.19600632e-02
 -1.28179946e-02 -8.47310410e-03 -6.04611864e-03 -4.54482565e-03
 -3.54804648e-03 -2.85080684e-03 -2.34313239e-03 -1.96152074e-03
 -1.66713344e-03 -1.43509056e-03 -1.24883409e-03 -1.09698597e-03
 -9.71511636e-04 -8.66602641e-04 -7.77973044e-04 -7.02402887e-04
 -6.37434169e-04 -5.81163632e-04 -5.32098601e-04 -4.89054827e-04
 -4.51082903e-04 -4.17414117e-04 -3.87422791e-04 -3.60587062e-04
 -3.36483586e-04 -3.14754625e-04 -2.95099914e-04 -2.77265942e-04
 -2.61037601e-04 -2.46231582e-04 -2.32691129e-04 -2.20281872e-04
 -2.08888537e-04 -1.98412389e-04 -1.88769355e-04 -1.79888807e-04
 -1.71713131e-04 -1.64198375e-04 -1.57316664e-04 -1.51061967e-04
 -1.45463118e-04 -1.40614814e-04 -1.36761330e-04 -1.34576978e-04
 -1.36555651e-04 -1.69697754e-04] 5
G_strang PCG time spend for n = 50, first toeplitz matrix : 0.0016021728515625 s
G_strang PCG time spend for n = 400, first toeplitz matrix : 0.0011661052703857422 s
G_strang PCG time 

We can see that the time consuming is a little bit small than CG, the main reason as I stated, it was because forming the inital matrix took a long time. And my computer for CG is very fast, I think if I have a worse computer, the difference bettween them will be very large. Right now because forming matrix takes so long, we can't see much difference. But we can see the itertion in PCG takes less time. In addition, if my coding skill is better, then I can write more efficient code to form initial matrix and preconditioners so that the difference will be better recognized.

Similiarly if we use T-chan preconditioner:

### PCG for toeplitz A in T-chan preconditoner

In [409]:
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:

    b = np.zeros(n)
    b[0] = 1
    A = toeplitz_1(n,p[0])
    A_increase = A_2n_fast(A)
    C = T_chan_fast(n,A)
    start = time.time()
    result,iteration = pcg_fft(A,n,A_increase,b,C)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'T_chan PCG time spend for n = {n}, first toeplitz matrix :', end - start,'s')
    else:
        print(f'T_chan PCG time spend for n = {n}, first toeplitz matrix :', end - start,'s')

[ 1.07071523e+00 -2.51649922e-01 -4.80172263e-02 -2.19600945e-02
 -1.28180115e-02 -8.47311065e-03 -6.04611822e-03 -4.54482063e-03
 -3.54803849e-03 -2.85079703e-03 -2.34312157e-03 -1.96150949e-03
 -1.66712218e-03 -1.43507960e-03 -1.24882366e-03 -1.09697625e-03
 -9.71502748e-04 -8.66594692e-04 -7.77966111e-04 -7.02397030e-04
 -6.37429433e-04 -5.81160051e-04 -5.32096196e-04 -4.89053599e-04
 -4.51082794e-04 -4.17415273e-04 -3.87423555e-04 -3.60591205e-04
 -3.36490064e-04 -3.14762773e-04 -2.95109236e-04 -2.77276045e-04
 -2.61048152e-04 -2.46242281e-04 -2.32701690e-04 -2.20292005e-04
 -2.08897931e-04 -1.98420699e-04 -1.88776184e-04 -1.79893694e-04
 -1.71715536e-04 -1.64197674e-04 -1.57312167e-04 -1.51053001e-04
 -1.45449279e-04 -1.40596691e-04 -1.36742600e-04 -1.34570924e-04
 -1.36606029e-04 -1.69683168e-04] 5
T_chan PCG time spend for n = 50, first toeplitz matrix : 0.002228975296020508 s
T_chan PCG time spend for n = 400, first toeplitz matrix : 0.0011932849884033203 s
T_chan PCG time spen

### PCG for toeplitz b G_strang

In [410]:
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:
   
    b = np.zeros(n)
    b[0] = 1
    A = toeplitz_b(n)
    A_increase = A_2n_fast(A)
    C = G_strang_fast(n,A)
    start = time.time()
    result,iteration = pcg_fft(A,n,A_increase,b,C)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'G_strang PCG time spend for n = {n}, second toeplitz matrix :', end - start,'s')
    else:
        print(f'G_strang PCG time spend for n = {n}, second toeplitz matrix :', end - start,'s')

[ 1.36698292e-01  1.51232059e-01  7.56490527e-02  2.22610546e-02
 -3.07094023e-03 -6.80919938e-03 -5.00468074e-03 -1.61251424e-03
 -3.69327417e-04  4.17415942e-04  1.54969824e-04  2.21669291e-04
 -4.69931464e-05  6.16473319e-05 -7.33135338e-05  4.48755858e-05
 -5.00582174e-05  4.19849705e-05 -3.74107037e-05  3.48302184e-05
 -3.10379261e-05  2.85736239e-05 -2.62104869e-05  2.40421013e-05
 -2.22701746e-05  2.05994720e-05 -1.91452274e-05  1.78379560e-05
 -1.66547616e-05  1.55930090e-05 -1.46278037e-05  1.37517960e-05
 -1.29536567e-05  1.22242246e-05 -1.15569971e-05  1.09431845e-05
 -1.03817673e-05  9.86220021e-06 -9.37974199e-06  8.95632908e-06
 -8.51301875e-06  8.17730860e-06 -7.85963734e-06  7.29595921e-06
 -7.65917158e-06  6.41682521e-06 -6.56706196e-06  9.31712827e-06
  1.29201770e-06  1.92362305e-05] 6
G_strang PCG time spend for n = 50, second toeplitz matrix : 0.0014541149139404297 s
G_strang PCG time spend for n = 400, second toeplitz matrix : 0.0015239715576171875 s
G_strang PCG 

### PCG for toeplitz b T_chan

In [411]:
n_matrix = [50,400,800,2000,4000,8000]
p = [2,1,1/10,1/100]

for n in n_matrix:
   
    b = np.zeros(n)
    b[0] = 1
    A = toeplitz_b(n)
    A_increase = A_2n_fast(A)
    C = T_chan_fast(n,A)
    start = time.time()
    result,iteration = pcg_fft(A,n,A_increase,b,C)
    end = time.time()
    # print(iteration)
    if n == 50:
        print(result, iteration)
        print(f'T_chan PCG time spend for n = {n}, second toeplitz matrix :', end - start,'s')
    else:
        print(f'T_chan PCG time spend for n = {n}, second toeplitz matrix :', end - start,'s')

[ 1.36698292e-01  1.51232064e-01  7.56490802e-02  2.22610756e-02
 -3.07094057e-03 -6.80919486e-03 -5.00468624e-03 -1.61250945e-03
 -3.69331114e-04  4.17416575e-04  1.54964422e-04  2.21669732e-04
 -4.69940491e-05  6.16501267e-05 -7.33128286e-05  4.48763240e-05
 -5.00592261e-05  4.19828174e-05 -3.74124449e-05  3.48292643e-05
 -3.10320638e-05  2.85866390e-05 -2.61876007e-05  2.40633921e-05
 -2.22587672e-05  2.06026496e-05 -1.91456412e-05  1.78364658e-05
 -1.66556671e-05  1.55925422e-05 -1.46278267e-05  1.37516345e-05
 -1.29538837e-05  1.22238311e-05 -1.15569120e-05  1.09440826e-05
 -1.03798352e-05  9.86403339e-06 -9.37985307e-06  8.95294502e-06
 -8.51858297e-06  8.17371683e-06 -7.85892554e-06  7.29996227e-06
 -7.65767426e-06  6.41834537e-06 -6.56563223e-06  9.31917351e-06
  1.29274257e-06  1.92361473e-05] 8
T_chan PCG time spend for n = 50, second toeplitz matrix : 0.0016071796417236328 s
T_chan PCG time spend for n = 400, second toeplitz matrix : 0.0015218257904052734 s
T_chan PCG time s

To conclude, we can see that when we use PCG for toeplitz system b, we get time increase $O(nlogn)$ which is smaller than the CG increase. 
